In [1]:
import os
from tasrif.processing_pipeline import SequenceOperator, ComposeOperator, MapIterableOperator
from tasrif.data_readers.fitbit_interday_dataset import FitbitInterdayDataset
from tasrif.processing_pipeline.pandas import ConvertToDatetimeOperator, SetIndexOperator, ConcatOperator, MergeOperator, \
AsTypeOperator, DropFeaturesOperator
from tasrif.processing_pipeline.custom import CreateFeatureOperator, AggregateOperator, FlattenOperator

In [2]:
interday_folder_path = os.environ.get('FITBIT_INTERDAY_PATH', '/mnt/data/fitbit-data/')
interday_folder_path

'/mnt/data/fitbit-data/'

In [4]:
import tasrif.yaml_parser as yaml_parser
import yaml    

In [5]:
with open("example.yaml", "r") as stream:
    try:
        p = yaml_parser.from_yaml(stream)
    except yaml.YAMLError as exc:
        print(exc)

TypeError: replace() argument 2 must be str, not None